# Part 2, Topic 1: Introduction to Voltage Glitching (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *Similarly to clock glitching, inserting brief glitches into the power line of an embedded device can result in skipped instructions and corrupted results. Besides providing a more reliable glitch on some targets when compared to clock glitching, voltage glitching also has the advanatage that the Vcc pins on chips are always accessable. This won't be covered in this course, but it can also be used to glitch a device asynchronous to its clock.*

**LEARNING OUTCOMES:**

* Understanding voltage glitch settings
* Building a voltage glitch and crash map.
* Modifying glitch circuit to increase glitch success

## Voltage Glitch Hardware

The ChipWhisperer uses the same hardware block for both voltage and clock glitching, with the only difference being where the glitch output is routed to. Instead of routing to HS2, voltage glitching is performed by routing the glitch to either the `glitch_hp` transistor or the `glitch_lp` transistor. This can be done via the following API calls:

```python
scope.io.glitch_hp = True #enable HP glitch
scope.io.glitch_hp = False #disable LP glitch
scope.io.glitch_lp = True #enable LP glitch
scope.io.glitch_lp = False #disable LP glitch
```

While the hardware block are the same, you'll need to change how it's configued. You wouldn't want to try routing `"clock_xor"` to the glitch transistor and oscillate Vcc like the device's clock! Instead, the following two output settings are best suited to voltage glitching:

1. `"glitch_only"` - insert a glitch for a portion of a clock cycle based on `scope.glitch.width` and `scope.glitch.offset`
1. `"enable_only"` - insert a glitch for an entire clock cycle

Typically, the `"enable_only"` setting will be too powerful for most devices. One situation where it outshines `"glitch_only"` is in glitching asychronous to the target's clock. An example of this is glitching a target with an internal clock. In this case, the ChipWhisperer's clock can be boosted far above the target's to insert a precise glitch, with `repeat` functioning as `width` and `ext_offset` functioning as `offset`.

### Voltage Glitching vs. Clock Glitching

Voltage glitching has some obvious benefits over clock glitching, such as working for a wider varitey of targets, but its downsides are less obvious. One of the biggest is how much it depends on the actual glitch circuit itself. With clock glitching, it's relatively easy to insert a glitch - there's nothing external trying to keep the clock at a certain voltage level. This is very different for a target's power pins. When we try to drop the power pin to ground, there's a lot of stuff fighting us to keep the power pin at the correct voltage, such as decoupling capacitors, bulk supply capacitors, and the voltage regulator supplying the voltage. This means when we make small changes to the glitch circuit, the glitch settings and even our ability to insert a glitch at all completely change! Consider glitching a target on the CW308 UFO board. If you switch your coaxial cable length from 20cm to 40 cm, you'll need to find entirely new glitch settings to repeat the attack (if it's still even possible). This is quite easy to see on an oscilloscope or  using the ChipWhisperer's ADC: longer cables and lower valued shunt resistors will make the glitch less sharp and increase ringing.

While your first thought might be to go for as sharp a glitch as possible, this often won't result in a high glitch success rate. If you're unable to find any working glitches with your current setup, it might be worth changing you hardware setup a bit. For example, on the ChipWhisperer Lite 1 part, you can desolder SJ5 and solder header pins to JP6. Even just connecting these pins with a jumper will have different glitch behaviour than with a soldered SJ5.

You can refer to the training slides for more information about finding good glitch settings, as well as more on the theory side of voltage glitching.

### The Lab

To introduce you to volatge glitching and find some settings, we're going to walk back through the clock glitching loop lab. You may want to capture some power traces while you're first experimenting with glitches to see what effects different glitch widths have on the power trace. Another thing to keep in mind is that targets often won't tolerate the Vcc pin dropping for an extended period of time without crashing - once you see the target start to crash, you won't see much else with larger widths.

One thing you might have to change is the glitch repeat value. Depending on how wide your glitch is, the voltage at the power pin may not recover by the time the next glitch is inserted. This can have to effect of increasing subsequent glitches' strength, which may or may not be desirable. Since glitches inserted with repeat > 1 have different strength, it's a good idea to scan through ext_offset as well.

###  Higher Frequency Glitching

The XMEGA target, and to a lesser extent the STM32F3, is very difficult to glitch with the default ChipWhisperer settings. Try bumping the clock frequency to 24MHz for the STM32 or 32MHz for the XMEGA and use a repeat 5-10 with both the high power and low power glitches active. You'll need to adjust the baud rate by the same proportion as the clock. This will increase the glitch precision, as the glitch width/offset step is based on the target clock, and may decrease the stability of the target.

Another setup that seems to work with the XMEGA is SJ5 unsoldered, JP6 jumpered, high+low power glitch, 32MHz, and repeat=5.

The ChipWhisperer Husky, with its PLL based glitching system, can typically glitch targets without needing to change the target's frequency.

### Disabling Logging

When glitching (or just running normally in earlier labs), you may have seen various warnings from loggers ChipWhisperer uses. This often has useful information, especially if things don't work right, but for voltage 
glitching especially, it mostly clutters up any print output you have. As such, we'll disable logging for the voltage glitching labs:

```python
cw.set_all_log_levels(cw.logging.CRITICAL)
```

You can reenable logging via

```python
cw.set_all_log_levels(cw.logging.WARNING)
```

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
SS_VER = 'SS_VER_2_1'

CRYPTO_TARGET = 'TINYAES128C'
allowable_exceptions = None
VERSION = 'HARDWARE'


In [2]:

#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope(hw_location=(5, 4))

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope(hw_location=(5, 4))
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")


# In[ ]:


if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
elif "neorv32" in PLATFORM.lower():
    prog = cw.programmers.NEORV32Programmer
elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
    prog = cw.programmers.SAM4SProgrammer
else:
    prog = None


# In[ ]:


import time
time.sleep(0.05)
scope.default_setup()

def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    elif "neorv32" in PLATFORM.lower():
        raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
    elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
        scope.io.nrst = 'low'
        time.sleep(0.25)
        scope.io.nrst = 'high_z'
        time.sleep(0.25)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)




INFO: Found ChipWhisperer😍


scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 94859030                  to 106092864                
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 14097003                  to 29538459                 
scope.clock.adc_rate                     changed from 14097003.0                to 29538459.0               
scope.clock.clkgen_

In [3]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../firmware/mcu/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

SS_VER set to SS_VER_2_1


SS_VER set to SS_VER_2_1


avr-gcc (GCC) 5.4.0
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the

 source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A

 PARTICULAR PURPOSE.



...

+ Specify buildtarget == allquick == to avoid full rebuild


+--------------------------------------------------------


+--------------------------------------------------------


+ Built for platform CW-Lite XMEGA with:


+ CRYPTO_TARGET = NONE
+ CRYPTO_OPTIONS = 
+--------------------------------------------------------

In [4]:
fw_path = "../../../firmware/mcu/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
if SS_VER=="SS_VER_2_1":
    target.reset_comms()

XMEGA Programming flash...


XMEGA Reading flash...
Verified flash OK, 2799 bytes


In [5]:
def reboot_flush():
    reset_target(scope)
    target.flush()

if scope._is_husky is False:
    if PLATFORM == "CWLITEXMEGA":
        scope.clock.clkgen_freq = 32E6
        if SS_VER=='SS_VER_2_1':
            target.baud = 230400*32/7.37
        else:
            target.baud = 38400*32/7.37
    elif (PLATFORM == "CWLITEARM") or ("F3" in PLATFORM):
        scope.clock.clkgen_freq = 24E6
        if SS_VER=='SS_VER_2_1':
            target.baud = 230400*24/7.37
        else:
            target.baud = 38400*24/7.37
        time.sleep(0.1)

In [6]:
reboot_flush()
scope.arm()
target.simpleserial_write("g", bytearray([]))
scope.capture()
val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)

{'valid': True, 'payload': CWbytearray(b'c4 09 00 00'), 'full_response': CWbytearray(b'00 72 04 c4 09 00 00 15 00'), 'rv': bytearray(b'\x00')}


In [7]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset", "tries"])
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

FloatSlider(value=0.0, continuous_update=False, description='tries setting:', disabled=True, max=10.0, readout…

In [8]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, bufferlen=int(10E6))

:DynamicMap   []
   :Overlay
      .Points.I  :Points   [width,offset]
      .Points.II :Points   [width,offset]

Next, we'll setup the glitch. The main differences here compared to clock glitching is the use of `scope.io.glitch_lp/hp`, which activates the transistors used for voltage, and `scope.glitch.output = 'glitch_only'`, which sends the glitch output to the transistors instead of xoring it with the clock like with clock glitching.

In [9]:
if scope._is_husky:
    scope.glitch.enabled = True
    scope.glitch.clk_src = "pll"
    scope.io.glitch_hp = True
    scope.io.glitch_lp = False
else:
    scope.glitch.clk_src = "clkgen" # set glitch input clock
scope.glitch.output = "glitch_only" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
if PLATFORM == "CWLITEXMEGA":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CWLITEARM":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CW308_STM32F3":
    scope.io.glitch_hp = True
    scope.io.glitch_lp = True

Some tips for finding good glitches:

1. There's a lot of stuff fighting our glitch this time - unlike the clock line, the Vcc rail isn't supposed to oscillate! As such shorter glitches will have no effect. Often, good widths will be just below when the target starts consistantly crashing, so a good strategy is to find the minimum width where crashes always happen, then backing the width off a bit.
1. The repeat parameter behaves very differently than with voltage glitching - at the boosted clock rate, the Vcc often won't recover before the next glitch. Try different repeat values as well.
1. We've built in a success/reset measurement into the glitch loop. Once you've found some glitch spots, this will help you evaluate which ones are best for your target.

It can take a very long time to do go through the full search space, so you may want to stop after you get a certain number of succeses. By default here, it will be 1, but you may want to change it to 10, 20, or go even higher.

In [10]:
MAX_SUCCESSES = 100000

In [11]:
num_tries = 1 # increase to get better glitch stats
gc.set_range("tries", 1, num_tries)
if scope._is_husky:
    gc.set_range("width", 1850, 3000)
    gc.set_range("offset", 1800, 3000)
    gc.set_global_step([50]) # reduce to fine tune glitching
    
    gc.set_step("ext_offset", 1)
    gc.set_range("ext_offset", 10, 50)
    scope.glitch.repeat = 1
    scope.adc.lo_gain_errors_disabled = True
    scope.adc.clip_errors_disabled = True
elif PLATFORM=="CWLITEXMEGA":
    gc.set_range("width", 43.5, 47.8)
    gc.set_range("offset", -48, -10)
    gc.set_range("ext_offset", 7, 10)
    gc.set_global_step(0.4)
    gc.set_step("ext_offset", 1)
    scope.glitch.repeat = 11
elif PLATFORM == "CWLITEARM":
    #should also work for the bootloader memory dump
    gc.set_range("width", 34, 36)
    gc.set_range("offset", -40, 10)
    gc.set_range("ext_offset", 4, 30)
    gc.set_global_step(0.4)
    gc.set_step("ext_offset", 1)
    scope.glitch.repeat = 7
elif PLATFORM == "CW308_STM32F3":
    #these specific settings seem to work well for some reason
    #also works for the bootloader memory dump
    gc.set_range("ext_offset", 4, 30)
    gc.set_range("width", 47.6, 49.6)
    gc.set_range("offset", -19, -21.5)
    gc.set_global_step(0.4)
    gc.set_step("ext_offset", 1)
    scope.glitch.repeat = 5

gc.set_step("tries", 1)

In [12]:
import struct

#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)

scope.adc.timeout = 0.5

reboot_flush()
loff = scope.glitch.offset
lwid = scope.glitch.width
total_successes = 0
successes = 0
resets = 0
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[2]
    #print(scope.glitch.ext_offset)
    if glitch_setting[3] == 1:
        total_successes += successes
        if (successes > 0):
            print("successes = {}, resets = {}, offset = {}, width = {}, ext_offset = {}".format(successes, resets, scope.glitch.offset, scope.glitch.width, scope.glitch.ext_offset))
            total_successes += successes
        successes = 0
        resets = 0
        #if total_successes > MAX_SUCCESSES:
        #    break
    target.flush()
    if scope.adc.state:
        # can detect crash here (fast) before timing out (slow)
        #print("Trigger still high!")
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
        resets += 1

    scope.arm()

    #Do glitch loop
    target.simpleserial_write("g", bytearray([]))

    ret = scope.capture()

    scope.io.vglitch_reset()
    if ret:
        #print('Timeout - no trigger')
        gc.add("reset")
        resets += 1

        #Device is slow to boot?
        reboot_flush()

    else:
        val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10, timeout=50)#For loop check
        if val['valid'] is False:
            gc.add("reset")
            reboot_flush()
            resets += 1
            #print(val)
        else:
            gcnt = struct.unpack("<I", val['payload'])[0]

            if gcnt != 2500: #for loop check
                gc.add("success")
                #print((scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
                successes += 1
            else:
                gc.add("normal")

print("Done glitching")
#enable logging
cw.set_all_log_levels(cw.logging.WARNING)

successes = 1, resets = 0, offset = -24.609375, width = 46.09375, ext_offset = 8


successes = 1, resets = 0, offset = -24.21875, width = 46.09375, ext_offset = 7


successes = 1, resets = 0, offset = -24.21875, width = 46.09375, ext_offset = 10


successes = 1, resets = 0, offset = -23.828125, width = 46.09375, ext_offset = 9


successes = 1, resets = 0, offset = -23.4375, width = 46.09375, ext_offset = 10


...

successes = 1, resets = 0, offset = -21.484375, width = 47.65625, ext_offset = 8


successes = 1, resets = 0, offset = -19.921875, width = 47.65625, ext_offset = 7


successes = 1, resets = 0, offset = -11.71875, width = 47.65625, ext_offset = 8


successes = 1, resets = 0, offset = -10.546875, width = 47.65625, ext_offset = 10


Done glitching


With that done, let's print our results. You'll likely want to ignore the "tries" and "ext_offset" parameters:

In [13]:
results = gc.calc(ignore_params=["tries", "ext_offset"], sort="success_rate")
results

[((47.09999999999999, -32.400000000000055),
  {'total': 4,
   'success': 4,
   'success_rate': 1.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 0,
   'normal_rate': 0.0}),
 ((47.09999999999999, -35.600000000000044),
  {'total': 4,
   'success': 4,
   'success_rate': 1.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 0,
   'normal_rate': 0.0}),
 ((46.69999999999999, -10.000000000000108),
  {'total': 4,
   'success': 4,
   'success_rate': 1.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 0,
   'normal_rate': 0.0}),
 ((46.69999999999999, -10.400000000000109),
  {'total': 4,
   'success': 4,
   'success_rate': 1.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 0,
   'normal_rate': 0.0}),
 ((46.69999999999999, -10.800000000000109),
  {'total': 4,
   'success': 4,
   'success_rate': 1.0,
   'reset': 0,
   'reset_rate': 0.0,
   'normal': 0,
   'normal_rate': 0.0}),
 ((46.69999999999999, -11.20000000000011),
  {'total': 4,
   'success': 4,
   'success_rate': 1.0,
   'reset':

In [14]:
gc.plot_2d(alpha=False)

[0, 1, 2, 3]
['width', 'offset']
(43.5, -48)


:Overlay
   .Points.I       :Points   [Width,Offset]
   .Points.Success :Points   [width,offset]   (success_rate)
   .Points.Reset   :Points   [width,offset]   (reset_rate)

In [15]:
scope.dis()
target.dis()

In [16]:
assert total_successes >= 1